<a href="https://colab.research.google.com/github/moqri/deep_cell/blob/master/starter_code/blood_cells.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#install required packages
!pip install scanpy

In [0]:
folder='kidney_fetal'
expressions_url='http://cells.ucsc.edu/kidney-atlas/Fetal_Full/exprMatrix.tsv.gz'
cells_url='http://cells.ucsc.edu/kidney-atlas/Fetal_Full/meta.tsv'

In [8]:
# Download the data 
!mkdir {folder}
!wget -O kidney_fetal/exprMatrix.tsv.gz {expressions_url}
!wget -O kidney_fetal/meta.tsv {cells_url}

mkdir: cannot create directory ‘kidney_fetal’: File exists
--2020-01-18 17:52:14--  http://cells.ucsc.edu/kidney-atlas/Fetal_Full/exprMatrix.tsv.gz
Resolving cells.ucsc.edu (cells.ucsc.edu)... 128.114.119.132, 128.114.119.131
Connecting to cells.ucsc.edu (cells.ucsc.edu)|128.114.119.132|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 365630999 (349M) [application/x-gzip]
Saving to: ‘kidney_fetal/exprMatrix.tsv.gz’

kidney_fetal/exprMa 100%[===================>] 348.69M  18.2MB/s    in 20s     

2020-01-18 17:52:35 (17.1 MB/s) - ‘kidney_fetal/exprMatrix.tsv.gz’ saved [365630999/365630999]

--2020-01-18 17:52:37--  http://cells.ucsc.edu/kidney-atlas/Fetal_Full/meta.tsv
Resolving cells.ucsc.edu (cells.ucsc.edu)... 128.114.119.132, 128.114.119.131
Connecting to cells.ucsc.edu (cells.ucsc.edu)|128.114.119.132|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2480230 (2.4M) [text/tab-separated-values]
Saving to: ‘kidney_fetal/meta.tsv’

kidney_

In [0]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import svm
import itertools
import numpy as np
import scanpy as sc
import pandas as pd

In [10]:
ad = sc.read_text(folder+"/exprMatrix.tsv.gz")
meta = pd.read_csv(folder+"/meta.tsv", sep="\t")
ad.var = meta
ad.data.shape

AnnData expects string indices for some functionality, but your first two indices are: RangeIndex(start=0, stop=2, step=1). 
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Use X instead of data, data will be removed in the future.
  after removing the cwd from sys.path.


(33694, 27203)

In [11]:
expression_df=pd.DataFrame(ad.data).transpose()
expression_df.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Use X instead of data, data will be removed in the future.
  """Entry point for launching an IPython kernel.


(27203, 33694)

In [12]:
expression_df.index.rename('Cell_ID',inplace=1)
expression_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,33654,33655,33656,33657,33658,33659,33660,33661,33662,33663,33664,33665,33666,33667,33668,33669,33670,33671,33672,33673,33674,33675,33676,33677,33678,33679,33680,33681,33682,33683,33684,33685,33686,33687,33688,33689,33690,33691,33692,33693
Cell_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,1.587717,0.0,0.0,0.0,0.0,0.0,0.0,...,3.865157,3.018610,0.0,3.192753,2.539844,3.470118,1.587717,2.807610,0.000000,0.0,2.539844,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500152,0.0,0.0,0.500152,0.000000,0.0,0.0,0.0,0.0,0.0,0.500152,0.0,0.0,0.0,0.0,0.0,0.0,...,3.892308,3.364041,0.0,2.726373,3.450040,3.080399,0.500152,3.244969,2.244599,0.0,3.018919,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,1.865399,1.865399,0.0,0.000000,1.865399,0.000000,0.000000,0.000000,0.000000,0.0,1.865399,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.010909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.628102,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,4.198009,3.304007,0.0,3.271371,3.857031,2.441355,0.628102,3.007122,1.010909,0.0,2.707036,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.628102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.108501,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,2.108501,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,5.479740,4.627934,0.0,3.793585,4.981672,3.944514,2.108501,4.295364,2.739018,0.0,3.944514,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
from sys import getsizeof
getsizeof(expression_array)

3127039368

In [61]:
cell_types=meta['celltype']
cell_df=pd.DataFrame(cell_types)
cell_df.index.rename('Cell_ID',inplace=1)
cell_df.columns=['Cell_Type']
cell_df.head()

,Cell_Type
Cell_ID,
0,NK cell
1,Stroma progenitor
2,Proximal tubule
3,Cap mesenchyme
4,Cap mesenchyme


In [14]:
cell_df['Cell_Type'].value_counts()

Stroma progenitor                     3841
Cap mesenchyme                        3558
Myofibroblast 1                       1744
Proliferating cap mesenchyme          1560
Macrophage 1                          1543
Endothelium                           1222
Monocyte                               783
Medial S shaped body                   742
Proliferating distal renal vesicle     725
Proliferating stroma progenitor        718
NK cell                                707
Proximal tubule                        707
Fibroblast 1                           674
Macrophage 2                           648
cDC2                                   646
Proximal S shaped body                 622
CNT/PC - proximal UB                   575
Distal S shaped body                   554
Distal renal vesicle                   494
Proliferating myofibroblast            470
Myofibroblast 2                        440
CD4 T cell                             438
Proximal UB                            405
Fibroblast 

In [0]:
expression_df=expression_df.loc[cell_df.index]
cell_df['Cell_Type'] = pd.Categorical(cell_df['Cell_Type'])
cell_df['Cell_Type'] = cell_df['Cell_Type'].cat.codes

In [0]:
expression_array=expression_df.values
expression_array=expression_array[:,~np.all(expression_array == 0, axis=0)] # remove all zero columns
expression_array=np.log(expression_array+1)
cell_array=cell_df.values

In [0]:
np.random.seed(1)
X_train, X_test, y_train, y_test = train_test_split(
    expression_array, cell_array, test_size=0.33, random_state=1)
#clf = svm.SVC(decision_function_shape='ovo')
#y_predict=clf.fit(X_train, y_train)
#y_predict=clf.predict(X_test)
#accuracy_score(y_test, y_predict)

In [0]:
#predictions=pd.DataFrame(data={'label':list(itertools.chain(*y_test)),'predicted':list(y_predict)})
#predictions.head()

In [0]:
%tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf

In [0]:
tf.random.set_seed(1)

In [30]:
dataset=tf.data.Dataset.from_tensor_slices((expression_array,cell_array ))
dataset

<TensorSliceDataset shapes: ((28738,), (1,)), types: (tf.float32, tf.int8)>

In [0]:
#for feat, targ in dataset.take(1):
#  print ('Features: {}, Target: {}'.format(feat, targ))

In [0]:
train_dataset = dataset.shuffle(len(cell_df)).batch(1)
cell_type_count=len(cell_df['Cell_Type'].value_counts())

In [0]:
def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(2000, activation='relu'),
    tf.keras.layers.Dense(cell_type_count, activation='softmax')
  ])

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [51]:
model = get_compiled_model()
model.fit(X_train, y_train, epochs=10)
model.evaluate(X_test,  y_test, verbose=2)

Train on 18226 samples
Epoch 1/10
18226/18226 [==============================] - 5s 283us/sample - loss: 0.6397 - accuracy: 0.8309
Epoch 2/10
18226/18226 [==============================] - 5s 273us/sample - loss: 0.0781 - accuracy: 0.9756
Epoch 3/10
18226/18226 [==============================] - 5s 274us/sample - loss: 0.0351 - accuracy: 0.9891
Epoch 4/10
18226/18226 [==============================] - 5s 273us/sample - loss: 0.0374 - accuracy: 0.9907
Epoch 5/10
18226/18226 [==============================] - 5s 275us/sample - loss: 0.0629 - accuracy: 0.9813
Epoch 6/10
18226/18226 [==============================] - 5s 273us/sample - loss: 0.0236 - accuracy: 0.9940
Epoch 7/10
18226/18226 [==============================] - 5s 275us/sample - loss: 0.0184 - accuracy: 0.9947
Epoch 8/10
18226/18226 [==============================] - 5s 275us/sample - loss: 0.0348 - accuracy: 0.9901
Epoch 9/10
18226/18226 [==============================] - 5s 273us/sample - loss: 0.0670 - accuracy: 0.9822
Epoch

[0.3336838413919215, 0.92180014]